In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="MAP"
cutoff_to_optimize=10

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommenderClass = ItemKNNCFRecommender



In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
rp3_study = optuna.load_study(study_name="RP3_Hybrid_last", storage=storage)
p3_study = optuna.load_study(study_name="P3_Hybrid_last", storage=storage)
slim_study = optuna.load_study(study_name="SLIM_Hybrid_last", storage=storage)
knn_study = optuna.load_study(study_name="KNN_Hybrid_last", storage=storage)


In [15]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.Hybrid.GeneralizedLinearHybridRecommender import GeneralizedLinearHybridRecommender

slims = []
rp3s = []
p3s = []
knns = []

for i in range(k_fold):
    slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train[i])
    slim.fit(workers=8, **slim_study.best_params)
    slims.append(slim)

    rp3 = RP3betaRecommender(urm_train[i])
    rp3.fit(**rp3_study.best_params)
    rp3s.append(rp3)

    p3 = P3alphaRecommender(urm_train[i])
    p3.fit(**p3_study.best_params)
    p3s.append(p3)

    knn = ItemKNNCFRecommender(urm_train[i])
    knn.fit(**knn_study.best_params)
    knns.append(knn)

SLIMElasticNetRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 332 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:42<00:00, 521.84it/s]


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6248.46 column/sec. Elapsed time 3.56 sec
P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 6362.82 column/sec. Elapsed time 3.49 sec
ItemKNNCFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 9365.18 column/sec. Elapsed time 2.37 sec
SLIMElasticNetRecommender: URM Detected 440 ( 3.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 325 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:43<00:00, 508.26it/s]


RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6407.20 column/sec. Elapsed time 3.47 sec
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 6574.12 column/sec. Elapsed time 3.38 sec
ItemKNNCFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
ItemKNNCFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 8997.02 column/sec. Elapsed time 2.47 sec
SLIMElasticNetRecommender: URM Detected 479 ( 3.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 345 ( 1.6%) items with no interactions.


...

RP3betaRecommender: URM Detected 471 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 362 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6650.53 column/sec. Elapsed time 3.34 sec
P3alphaRecommender: URM Detected 471 ( 3.7%) users with no interactions.
P3alphaRecommender: URM Detected 362 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 6937.01 column/sec. Elapsed time 3.20 sec
ItemKNNCFRecommender: URM Detected 471 ( 3.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 362 ( 1.6%) items with no interactions.
Similarity column 22222 (100.0%), 12553.10 column/sec. Elapsed time 1.77 sec
SLIMElasticNetRecommender: URM Detected 478 ( 3.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 329 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:47<00:00, 471.43it/s]


RP3betaRecommender: URM Detected 478 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6190.93 column/sec. Elapsed time 3.59 sec
P3alphaRecommender: URM Detected 478 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 6536.72 column/sec. Elapsed time 3.40 sec
ItemKNNCFRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 329 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 12700.02 column/sec. Elapsed time 1.75 sec
SLIMElasticNetRecommender: URM Detected 460 ( 3.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 348 ( 1.6%) items with no interactions.


100%|█████████▉| 22216/22222 [00:48<00:00, 454.30it/s]


RP3betaRecommender: URM Detected 460 ( 3.6%) users with no interactions.
RP3betaRecommender: URM Detected 348 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6092.13 column/sec. Elapsed time 3.65 sec
P3alphaRecommender: URM Detected 460 ( 3.6%) users with no interactions.
P3alphaRecommender: URM Detected 348 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 7625.98 column/sec. Elapsed time 2.91 sec
ItemKNNCFRecommender: URM Detected 460 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 348 ( 1.6%) items with no interactions.
Similarity column 22222 (100.0%), 12379.04 column/sec. Elapsed time 1.80 sec


In [16]:
hybrids = []

for i in range(k_fold):
    hybrid = GeneralizedLinearHybridRecommender(urm_train[i], recommenders=[slims[i], rp3s[i], p3s[i], knns[i]])
    hybrids.append(hybrid)

SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 332 ( 1.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 440 ( 3.5%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 325 ( 1.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 479 ( 3.8%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 345 ( 1.6%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 451 ( 3.6%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 311 ( 1.4%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFHybridRecommender: URM Detected 449 ( 3.6%) users with no interactions.
SLIMElasticNetRP3be

In [17]:
study = optuna.create_study(study_name="LINEAR_4_Hybrid_last", direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

[I 2024-01-10 13:01:33,082] A new study created in RDB with name: LINEAR_4_Hybrid_last


In [18]:
def objective(trial):

    alphas = []
    w_slim = trial.suggest_float("w_slim", 0.75, 1)
    w_knn = trial.suggest_float("w_knn", 0, 0.1)
    w_p3 = trial.suggest_float("w_p3", 0, 0.1)
    w_rp3 = trial.suggest_float("w_rp3", 0.35, 0.75)
        
    alphas = [w_slim, w_rp3, w_p3, w_knn]

    MAP = 0
    for i in range(k_fold):
        recommender = hybrids[i]
        recommender.fit(alphas=alphas)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)
        print(f"MAP: {result_dict['MAP'].item()}, RECALL: {result_dict['RECALL'].item()}")
        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    return MAP

In [ ]:
0.03112869156762209 #CON TOPPOP

0.031013042810739334 #SENZA TOPPOP

In [21]:
study.optimize(objective, n_trials=200)

EvaluatorHoldout: Processed 10057 (100.0%) in 6.83 sec. Users per second: 1472
MAP: 0.030899499835067283, RECALL: 0.11836124402656223
EvaluatorHoldout: Processed 10040 (100.0%) in 7.04 sec. Users per second: 1426
MAP: 0.03101209843167038, RECALL: 0.11741155274604338
EvaluatorHoldout: Processed 10017 (100.0%) in 6.99 sec. Users per second: 1434
MAP: 0.030538683444493727, RECALL: 0.11584664409269313
EvaluatorHoldout: Processed 10037 (100.0%) in 5.89 sec. Users per second: 1704
MAP: 0.030930889043871316, RECALL: 0.11921937218318204
EvaluatorHoldout: Processed 9989 (100.0%) in 6.72 sec. Users per second: 1487
MAP: 0.031106828622596158, RECALL: 0.12326075820682202
EvaluatorHoldout: Processed 10024 (100.0%) in 6.71 sec. Users per second: 1495
MAP: 0.03128578712676905, RECALL: 0.11859559219415794
EvaluatorHoldout: Processed 10030 (100.0%) in 7.06 sec. Users per second: 1420
MAP: 0.03127620709300698, RECALL: 0.12250073682608799
EvaluatorHoldout: Processed 10052 (100.0%) in 6.78 sec. Users per 

[I 2024-01-10 14:04:45,026] Trial 46 finished with value: 0.031013042810739334 and parameters: {'w_slim': 0.8856705402423755, 'w_knn': 0.09599696002268149, 'w_p3': 0.024430537906598455, 'w_rp3': 0.6810149971480753}. Best is trial 22 with value: 0.031086245251289845.


EvaluatorHoldout: Processed 10057 (100.0%) in 5.88 sec. Users per second: 1711
MAP: 0.030848248317921347, RECALL: 0.11852922711131766
EvaluatorHoldout: Processed 10040 (100.0%) in 6.36 sec. Users per second: 1578
MAP: 0.031122739201922715, RECALL: 0.11797065543985084
EvaluatorHoldout: Processed 10017 (100.0%) in 6.97 sec. Users per second: 1437
MAP: 0.03062317869146287, RECALL: 0.1158181269448055
EvaluatorHoldout: Processed 10037 (100.0%) in 7.19 sec. Users per second: 1396
MAP: 0.03095388333009158, RECALL: 0.11935219893924913
EvaluatorHoldout: Processed 9989 (100.0%) in 6.76 sec. Users per second: 1479
MAP: 0.031213167817933308, RECALL: 0.1234982257052479
EvaluatorHoldout: Processed 10024 (100.0%) in 6.86 sec. Users per second: 1461
MAP: 0.031357678080543426, RECALL: 0.11907901290200877
EvaluatorHoldout: Processed 10030 (100.0%) in 7.01 sec. Users per second: 1431
MAP: 0.0314710946525506, RECALL: 0.12311747897598639
EvaluatorHoldout: Processed 10052 (100.0%) in 7.07 sec. Users per sec

[I 2024-01-10 14:06:05,257] Trial 47 finished with value: 0.031079677397496026 and parameters: {'w_slim': 0.7743726157285975, 'w_knn': 0.08882374664599411, 'w_p3': 0.05422828936846578, 'w_rp3': 0.7295071413456233}. Best is trial 22 with value: 0.031086245251289845.


EvaluatorHoldout: Processed 10057 (100.0%) in 7.10 sec. Users per second: 1416
MAP: 0.030962813550066408, RECALL: 0.11813245559162602
EvaluatorHoldout: Processed 10040 (100.0%) in 6.89 sec. Users per second: 1457
MAP: 0.03101220514766364, RECALL: 0.11754195236832136
EvaluatorHoldout: Processed 10017 (100.0%) in 6.87 sec. Users per second: 1457
MAP: 0.030529500642558598, RECALL: 0.11604137229752588
EvaluatorHoldout: Processed 10037 (100.0%) in 6.95 sec. Users per second: 1444
MAP: 0.030899971692041388, RECALL: 0.11932022747634799
EvaluatorHoldout: Processed 9989 (100.0%) in 6.90 sec. Users per second: 1447
MAP: 0.031046774467787815, RECALL: 0.12304341188125108
EvaluatorHoldout: Processed 10024 (100.0%) in 6.76 sec. Users per second: 1482
MAP: 0.03132371198013681, RECALL: 0.11893848123713066
EvaluatorHoldout: Processed 10030 (100.0%) in 5.76 sec. Users per second: 1741
MAP: 0.031142813622624457, RECALL: 0.12266856667460688
EvaluatorHoldout: Processed 10052 (100.0%) in 6.95 sec. Users per

...

EvaluatorHoldout: Processed 10057 (100.0%) in 4.92 sec. Users per second: 2044
MAP: 0.03090923403268058, RECALL: 0.11869332396894111
EvaluatorHoldout: Processed 10040 (100.0%) in 5.34 sec. Users per second: 1880
MAP: 0.031187029659141517, RECALL: 0.11844272684205175
EvaluatorHoldout: Processed 10017 (100.0%) in 5.38 sec. Users per second: 1863
MAP: 0.030580422012737335, RECALL: 0.1157095726501557
EvaluatorHoldout: Processed 10037 (100.0%) in 5.40 sec. Users per second: 1858
MAP: 0.03091849442776042, RECALL: 0.11936118832833666
EvaluatorHoldout: Processed 9989 (100.0%) in 5.46 sec. Users per second: 1829
MAP: 0.031163823857036626, RECALL: 0.12362409397822599
EvaluatorHoldout: Processed 10024 (100.0%) in 5.29 sec. Users per second: 1894
MAP: 0.03143151278202176, RECALL: 0.1192468438929453
EvaluatorHoldout: Processed 10030 (100.0%) in 5.41 sec. Users per second: 1853
MAP: 0.03146469717830666, RECALL: 0.12327162462272259
EvaluatorHoldout: Processed 10052 (100.0%) in 5.53 sec. Users per sec

[I 2024-01-10 17:05:44,977] Trial 200 finished with value: 0.03110901689460417 and parameters: {'w_slim': 0.7603119797403111, 'w_knn': 0.09024209638198831, 'w_p3': 0.09588019146654307, 'w_rp3': 0.7397417497197055}. Best is trial 183 with value: 0.031120660183068433.


EvaluatorHoldout: Processed 10057 (100.0%) in 5.39 sec. Users per second: 1865
MAP: 0.030921106833904102, RECALL: 0.11870964700326249
EvaluatorHoldout: Processed 10040 (100.0%) in 5.15 sec. Users per second: 1948
MAP: 0.031040607411623664, RECALL: 0.1179087589047882
EvaluatorHoldout: Processed 10017 (100.0%) in 4.92 sec. Users per second: 2037
MAP: 0.030557896813512442, RECALL: 0.11582144102857714
EvaluatorHoldout: Processed 10037 (100.0%) in 5.46 sec. Users per second: 1838
MAP: 0.03089657552769071, RECALL: 0.11938249656564474
EvaluatorHoldout: Processed 9989 (100.0%) in 5.49 sec. Users per second: 1819
MAP: 0.031098740360428395, RECALL: 0.1230800988299358
EvaluatorHoldout: Processed 10024 (100.0%) in 5.35 sec. Users per second: 1873
MAP: 0.031338244562257166, RECALL: 0.1194501950682539
EvaluatorHoldout: Processed 10030 (100.0%) in 5.40 sec. Users per second: 1859
MAP: 0.031294188861985754, RECALL: 0.12312696744150786
EvaluatorHoldout: Processed 10052 (100.0%) in 5.41 sec. Users per s

[I 2024-01-10 17:06:52,364] Trial 201 finished with value: 0.031029546189969316 and parameters: {'w_slim': 0.7552618891241746, 'w_knn': 0.08982118342424836, 'w_p3': 0.097790925614128, 'w_rp3': 0.4910064540348884}. Best is trial 183 with value: 0.031120660183068433.


EvaluatorHoldout: Processed 10057 (100.0%) in 5.28 sec. Users per second: 1905
MAP: 0.03089698243819771, RECALL: 0.11862726608768723
EvaluatorHoldout: Processed 10040 (100.0%) in 5.26 sec. Users per second: 1907
MAP: 0.031189171883893297, RECALL: 0.11840400461949133
EvaluatorHoldout: Processed 10017 (100.0%) in 5.30 sec. Users per second: 1888
MAP: 0.030586261292311265, RECALL: 0.11590609795296725
EvaluatorHoldout: Processed 10037 (100.0%) in 5.29 sec. Users per second: 1898
MAP: 0.030921617791947846, RECALL: 0.11944185890919513


In [ ]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

In [ ]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train_validation[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test